In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [2]:
cd MyDrive/MyDrive/deepLearning/dataset

/content/MyDrive/MyDrive/deepLearning/dataset


In [3]:
import pandas as pd
# 각 subreddit 내 documents를 fast text로 변환하고 train, test로 나눈 데이터
train = pd.read_csv('mental_fast_train.csv')
test = pd.read_csv('mental_fast_test.csv')

In [4]:
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,class
0,-0.384500,0.082557,0.099619,-0.142753,-0.121291,0.160080,-0.313860,-0.016713,-0.275874,-0.265333,-0.023278,-0.001175,-0.139884,-0.093425,-0.154021,-0.318769,0.031404,-0.256025,-0.269931,-0.068348,0.141830,-0.034978,0.011819,0.083629,-0.077831,-0.055735,-0.211576,0.031547,-0.269471,0.106786,-0.281489,-0.006999,-0.128658,-0.129168,0.103264,0.095905,-0.028059,0.061610,-0.140036,-0.075074,...,0.154353,0.051405,-0.013978,-0.092510,-0.209597,-0.034280,0.098459,0.130065,0.025791,-0.159216,-0.056381,-0.190561,0.058225,0.088167,-0.037777,-0.211007,-0.122538,-0.184446,-0.189247,-0.077104,0.063776,0.196836,-0.129735,0.307711,-0.155830,-0.353103,-0.025074,-0.068467,0.221661,-0.522361,-0.029648,-0.090275,-0.012135,-0.093605,0.044153,0.243966,-0.148416,-0.040172,0.289762,5
1,-0.297447,-0.103995,0.128785,-0.130750,-0.139462,0.203264,-0.208423,0.067351,-0.337381,-0.177630,-0.068390,0.038023,-0.078778,-0.229576,-0.141029,-0.306194,-0.065938,-0.315844,-0.170987,-0.096183,0.157459,0.063214,0.001203,0.195154,-0.098127,-0.130874,-0.187100,0.014191,-0.283526,0.135973,-0.299896,-0.162731,-0.201831,-0.182047,0.048330,0.012602,0.032556,-0.061304,-0.116796,0.044928,...,0.205597,0.088661,-0.167728,-0.068121,-0.209554,0.050250,0.090301,-0.001048,0.107183,-0.162686,-0.021244,-0.073225,0.111458,0.064623,-0.080862,-0.216198,-0.159353,-0.186579,-0.149208,-0.056262,0.050866,0.130655,-0.161180,0.265045,-0.080817,-0.284756,0.048123,-0.028504,0.191598,-0.466767,-0.092783,-0.093828,-0.030692,-0.074476,0.082181,0.251570,-0.174057,-0.153086,0.272814,2
2,-0.320336,0.027078,0.073990,-0.190082,-0.094764,0.119061,-0.253249,-0.002594,-0.311685,-0.178904,-0.101278,0.012539,-0.023853,-0.068392,-0.147039,-0.307807,-0.109156,-0.279428,-0.092608,-0.085723,0.099343,0.086204,0.070592,0.107499,-0.109580,-0.057108,-0.198804,0.005818,-0.263149,0.153789,-0.227880,-0.080192,-0.126785,-0.236534,0.064021,0.079126,0.011743,-0.056335,-0.091427,0.026182,...,0.217325,0.048513,-0.106926,-0.094417,-0.302636,0.054053,0.148431,0.075958,0.000304,-0.135758,-0.027215,-0.147851,0.126128,0.102037,0.019624,-0.256065,-0.163480,-0.259983,-0.218456,-0.089114,-0.028897,0.101531,-0.155858,0.274921,-0.072629,-0.345323,0.030471,0.069080,0.250834,-0.489751,-0.032385,-0.049597,-0.107659,0.014696,0.064800,0.269899,-0.246107,-0.054382,0.257916,1
3,-0.314621,-0.103269,0.267915,-0.072211,-0.032262,0.049409,-0.302219,0.010292,-0.325797,-0.070215,-0.291989,0.262244,-0.059147,-0.097145,-0.225114,-0.514871,0.090711,-0.433810,-0.127416,-0.059960,0.109920,0.069039,0.067691,0.267718,-0.207631,-0.081226,0.010850,-0.083392,-0.295549,0.159784,-0.163304,-0.017860,-0.103177,-0.093840,0.107396,0.000896,0.120507,-0.053836,-0.168668,-0.029859,...,0.247943,-0.002920,-0.014077,-0.129284,-0.005158,0.222307,0.062785,-0.057913,0.254240,-0.241263,-0.013552,0.045939,0.025222,-0.022215,-0.132905,-0.217853,-0.113178,-0.138009,-0.382347,0.046695,-0.052101,-0.017406,-0.110045,0.385967,-0.101195,-0.393539,0.128723,-0.019644,0.234558,-0.497922,-0.013003,-0.045641,-0.059600,-0.072691,0.129254,0.347997,0.052199,-0.083907,0.381378,0
4,-0.305389,-0.085013,0.083076,-0.136057,-0.123276,0.183976,-0.248987,0.021846,-0.324305,-0.207632,-0.072087,0.113956,-0.109851,-0.145616,-0.045205,-0.227112,-0.089186,-0.265792,-0.195291,-0.110115,0.146180,0.031207,0.022880,0.192689,-0.058824,-0.033871,-0.211157,0.021326,-0.328413,0.077824,-0.217231,-0.072830,-0.230071,-0.147024,0.085625,0.007216,-0.007075,0.004788,-0.185567,-0.025732,...,0.247570,-0.013305,-0.109231,-0.109628,-0.245428,0.064559,0.132149,-0.074365,-0.014154,-0.167515,-0.036473,-0.114275,0.122631,0.148622,-0.082316,-0.162193,-0.156540,-0.206708,-0.142941,0.018769,0.055051,0.162109,-0.183060,0.319636,-0.082537,-0.301412,0.144930,0.09

In [6]:
pip install http://h2o-release.s3.amazonaws.com/h2o/master/5387/Python/h2o-3.33.1.5387-py2.py3-none-any.whl

     |████████████████████████████████| 164.8 MB 1.4 MB/s 


In [7]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsdyd3anu
  JVM stdout: /tmp/tmpsdyd3anu/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpsdyd3anu/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.33.1.5387
H2O_cluster_version_age:,4 months and 27 days !!!
H2O_cluster_name:,H2O_from_python_unknownUser_39rnuo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [8]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
train['class'] = train['class'].asfactor()
test['class'] = test['class']

In [10]:
predictors = train.names[:-1]
response = 'class'

In [11]:
from h2o.automl import H2OAutoML
import time
start = time.time()
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=30,seed = 1,exclude_algos=['XGBoost'],max_runtime_secs = 1200,nfolds=3,keep_cross_validation_predictions=True,keep_cross_validation_models=True,keep_cross_validation_fold_assignment=True)
aml.train(x=predictors, y=response, training_frame=train,leaderboard_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
StackedEnsemble_AllModels_AutoML_20210821_132606,0.240618,0.721327,0.48304,0.233328,nan,nan
StackedEnsemble_BestOfFamily_AutoML_20210821_132606,0.244545,0.728354,0.48402,0.234276,nan,nan
DeepLearning_1_AutoML_20210821_132606,0.248853,0.742043,0.47777,0.228264,nan,nan
DeepLearning_grid__2_AutoML_20210821_132606_model_1,0.269714,0.849167,0.486395,0.23658,nan,nan
DeepLearning_grid__1_AutoML_20210821_132606_model_1,0.27436,0.976158,0.48949,0.2396,nan,nan
GBM_grid__1_AutoML_20210821_132606_model_1,0.310676,0.941678,0.578286,0.334415,nan,nan
DeepLearning_grid__3_AutoML_20210821_132606_model_1,0.324394,1.09139,0.639037,0.408369,nan,nan
GBM_3_AutoML_20210821_132606,0.352482,1.31616,0.716704,0.513665,nan,nan
GBM_2_AutoML_20210821_132606,0.361561,1.33572,0.722772,0.5224,nan,nan
GBM_5_AutoML_20210821_132606,0.369458,1.44767,0.756881,0.572868,nan,nan


In [13]:
from sklearn.metrics import classification_report
import numpy as np
Pred = h2o.as_list(aml.predict(test),use_pandas=False)
result = [i[0] for i in Pred]
result = result[1:]
test_label = np.array(h2o.as_list(test['class'],use_pandas=False)[1:]).reshape(1,-1)[0]
print(classification_report(test_label, result))

stackedensemble prediction progress: |████████████████████████████████████| 100%
              precision    recall  f1-score   support

           0       0.82      0.87      0.85      3947
           1       0.67      0.75      0.71      3947
           2       0.75      0.76      0.75      3947
           3       0.72      0.68      0.70      3946
           4       0.85      0.78      0.81      3947
           5       0.77      0.72      0.74      3947

    accuracy                           0.76     23681
   macro avg       0.76      0.76      0.76     23681
weighted avg       0.76      0.76      0.76     23681

